# MNIST and LeNet

In [0]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import copy
from torchsummary import summary
!pip install poutyne


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device", device)



     |████████████████████████████████| 92kB 3.1MB/s 
Using device cuda


# PyTorch Datasets and Dataloaders

Datasets documentation:

https://pytorch.org/docs/stable/data.html

Dataloaders

https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader


TorchVision datasets:

https://pytorch.org/docs/stable/torchvision/datasets.html

https://pytorch.org/docs/stable/_modules/torchvision/datasets/folder.html#ImageFolder


torchvision transforms:

https://pytorch.org/docs/stable/torchvision/transforms.html#generic-transforms

Example of data augmentation using transforms:

<code>
composed = transforms.Compose([
        Rescale(256),
        RandomCrop(224),
        transforms.RandomHorizontalFlip()
        ])
</code>

In [0]:
# Normalization, data transformation can also be used for data augmentation
transform = transforms.Compose([transforms.ToTensor(), 
                                        transforms.Normalize((0.5,), (0.5,))])

dataset = datasets.MNIST(root = './data', train = True, transform = transform, download=True)
train_set, val_set = torch.utils.data.random_split(dataset, [50000, 10000])

test_set = datasets.MNIST(root='./data', train = False, download = True, transform = transform)

batch_size = 50
train_loader = torch.utils.data.DataLoader(
                 dataset=dataset,
                 batch_size=batch_size,
                 shuffle=True, num_workers=2)

val_loader = torch.utils.data.DataLoader(
                dataset=val_set,
                batch_size=batch_size,
                shuffle=False, num_workers=2)


test_loader = torch.utils.data.DataLoader(
                dataset=test_set,
                batch_size=batch_size,
                shuffle=False, num_workers=2)


  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 18532034.36it/s]                            


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


32768it [00:00, 300106.67it/s]                           
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


1654784it [00:00, 5243217.45it/s]                           
8192it [00:00, 126734.92it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw
Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


Same MLPClassifier we saw last week

In [0]:
class MLPClassifier(nn.Module):
    def __init__(self, dim_in=28*28, num_classes=10):
        super(MLPClassifier, self).__init__()
        self.classifier = nn.Sequential( 
          nn.Linear(dim_in, 128),
          nn.ReLU(inplace=True),
          nn.Linear(128, num_classes),
        )
        
    # Not necessary to define backward, Autograd takes care of it
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        return  self.classifier(x)


    def raw_to_probs(self, raw):
        return torch.nn.functional.softmax(raw)

    def accuracy(self, predictions, y_true):
        y_pred = predictions.argmax(dim=1)
        #print(y_pred.cpu().numpy())
        acc_pred = (y_pred == y_true).float().mean()
        return acc_pred * 100


# Task

Write a LeNet-like Convolutional Neural Network
<img src="https://www.researchgate.net/profile/Yiren_Zhou/publication/312170477/figure/fig1/AS:448817725218816@1484017892071/Structure-of-LeNet-5.png">

Where it should have 
1.   2D Conv layer of 20 filters of 5x5 https://pytorch.org/docs/stable/nn.html#conv2d
2.   ReLU https://pytorch.org/docs/stable/nn.html#relu
3.   MaxPooling2D https://pytorch.org/docs/stable/nn.html#maxpool2d
4.   2D Conv layer of 50 filters of 5x5
5.   ReLU
6.   MaxPooling2D
7.   Flatten https://pytorch.org/docs/stable/nn.html#flatten
8.   Linear layer with 500 outputs https://pytorch.org/docs/stable/nn.html#linear
9.   ReLU
10.  Linear leyer with 10 output classes

It is easier to group convolutions, relus and maxpoolings using nn.Sequential https://pytorch.org/docs/stable/nn.html#sequential as a trainable feature extractor, and same for linear layers as the classifier


In [0]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.featurizer = nn.Sequential( #input image 1 channel 28 pixels width and 28 pixels height 1x28x28
            nn.Conv2d(1, 20, 5),  #output shape is 20x24x24
            nn.ReLU(),
            nn.MaxPool2d(2), #output shape is 20x12x12
            nn.Conv2d(20, 50, 5), #output shape is 50x8x8
            nn.ReLU(),
            nn.MaxPool2d(2), #output shape is 50x4x4
            nn.Flatten() #output shape is 1x800  (50*4*4)
        )

        self.classifier = nn.Sequential(
            nn.Linear(4*4*50, 500),
            nn.ReLU(),
            nn.Linear(500, 10)
        )

    def forward(self, x):
        x = self.featurizer(x)
        x = self.classifier(x)
        return x
    
    def raw_to_probs(self, raw):
        return torch.nn.functional.softmax(raw)

    def accuracy(self, predictions, y_true):
        y_pred = predictions.argmax(dim=1)
        acc_pred = (y_pred == y_true).float().mean()
        return acc_pred * 100


In [0]:
from poutyne.framework import Model
                      
#mymodel = MLPClassifier()
mymodel = LeNet()
mymodel = mymodel.to(device)
print(mymodel)
summary(mymodel, input_size=(1, 28, 28)) 
learning_rate = 0.01

# Optimizer and loss function
optimizer = optim.Adam( filter(lambda p: p.requires_grad, mymodel.parameters()), lr=learning_rate )
loss_function = nn.CrossEntropyLoss()

model = Model(mymodel, optimizer, loss_function, batch_metrics=['accuracy'], epoch_metrics=['f1'])

# Send model on GPU
model.to(device)

model.fit_generator(train_loader, val_loader, epochs=10)


 # Test
test_loss, test_acc = model.evaluate_generator(test_loader)
print('Test:\n\tLoss: {}\n\tAccuracy: {}'.format(test_loss, test_acc))

LeNet(
  (featurizer): Sequential(
    (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten()
  )
  (classifier): Sequential(
    (0): Linear(in_features=800, out_features=500, bias=True)
    (1): ReLU()
    (2): Linear(in_features=500, out_features=10, bias=True)
  )
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 20, 24, 24]             520
              ReLU-2           [-1, 20, 24, 24]               0
         MaxPool2d-3           [-1, 20, 12, 12]               0
            Conv2d-4             [-1, 50, 8, 8]          25,050
              ReLU-5             [-1, 50, 8, 8]               0
   

In [0]:
def eval_model(model, data_loader, criterion, acc_metric):
    model.eval()

    total_loss = 0
    total_acc = 0

    n_iter = 0
    with torch.no_grad():
        for data in data_loader:
            data, label = data
            if next(model.parameters()).is_cuda:
                data = data.to(device)
                label = label.to(device)

            raw_preds = model(data)
            loss = criterion(raw_preds, label).cpu().item()
            acc = acc_metric( raw_preds , label ).cpu().item()
            total_loss += loss
            total_acc += acc
            n_iter += 1


    total_loss /= n_iter
    total_acc /= n_iter
    model.train()
    return total_loss, total_acc



In [0]:

n_epochs = 15
model = LeNet()
#model =MLPClassifier()
model.to(device)
print(model)
summary(model, input_size=(1, 28,28))
learning_rate = 0.03
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss().to(device)


prev_loss = None
for epoch in range(n_epochs):

    model.train()
    train_losses =[]
    train_accs = []

    for i, data in enumerate( train_loader):
        data, label = data
        data = data.to(device)
        label = label.to(device)

        raw_preds = model(data)
        loss = criterion(raw_preds, label)
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        loss = loss.cpu().item()
        acc = model.accuracy( raw_preds , label ).cpu().item()
        train_losses.append(loss)
        train_accs .append(acc)
        if i%20 == 0:
            print("\r", "Batch:", i, "Training:", loss, " / ", " Training Acc:", acc, end="")            


    #Print losses and metrics every epoch
    train_loss = np.mean(np.asarray(train_losses))
    train_acc = np.mean(np.asarray(train_accs))
    val_loss, val_acc = eval_model(model, val_loader, criterion, model.accuracy)
    print("\n", "Epoch:", epoch, "Training/Validation Loss:", train_loss, " / ", val_loss, " Training/Valitaion Acc:", train_acc, " / ", val_acc)

    if prev_loss is None or val_loss < prev_loss:
        prev_loss = val_loss
        best_model = copy.deepcopy(model)

test_loss, test_acc = eval_model(model, test_loader, criterion, model.accuracy)
print("Testing Loss:", test_loss, " Testing Acc:", test_acc)
test_loss, test_acc = eval_model(best_model, test_loader, criterion, model.accuracy)
print("Testing Loss Best Model:", test_loss, " Testing Acc Best Model:", test_acc)

torch.save(best_model.state_dict(), "./best_model.pth")


LeNet(
  (featurizer): Sequential(
    (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten()
  )
  (classifier): Sequential(
    (0): Linear(in_features=800, out_features=500, bias=True)
    (1): ReLU()
    (2): Linear(in_features=500, out_features=10, bias=True)
  )
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 20, 24, 24]             520
              ReLU-2           [-1, 20, 24, 24]               0
         MaxPool2d-3           [-1, 20, 12, 12]               0
            Conv2d-4             [-1, 50, 8, 8]          25,050
              ReLU-5             [-1, 50, 8, 8]               0
   

In [0]:
model = LeNet()
model.load_state_dict(torch.load("./best_model.pth", map_location=torch.device('cpu')))
model.eval()
test_loss, test_acc = eval_model(model, test_loader, criterion, model.accuracy)
print("Testing Loss Best Model:", test_loss, " Testing Acc Best Model:", test_acc)

Testing Loss Best Model: 0.02293997878149753  Testing Acc Best Model: 99.28
